<a href="https://colab.research.google.com/github/HardikPaliwal/CS484Proj/blob/master/cs484%20proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CS484 Final Project**

#### Topic 6: Weakly supervised classification

#### Hardik Paliwal (20725413), Lance Pereira (20719626)

______________________________________________________________

#**Table Of Contents**
- A) Abstract
- B) High Level Goals and Methedology
- C) Team Members and Contributions
- D) External Code Libraries
- E) Code
  - 1) Setup (imports, loading data)
  - 2) Define our Base CNN (Based on VGG11), Test, Train methods
  - 3) Split Fashion MNIST training data into M labeled images, and N-M unlabeled
  - 4) Train Base CNN on M labeled training images
  - 5) Use the 'semi' trained CNN to gain important features (feature embedings) of N-M *unlabeled* training images
  - 6) Code: Assign labels to unlabeled images
  - 7) Define retrained CNN function using predicted labels from clustering for unlabeled data
- F) Experiments
  - 1) Diffirent ratios of M/N
  - 2) Diffirent Clustering Methods
- G) Results
  - 1) Results of diffirent ratios
  - 2) Results from diffirent clustering methods
  - 3) Results from Clustering vs retrained CNN
- H) Conclusions

#**A) Abstract:**

For our project we have decided to do Project 6, choosing specifically Fashion MNIST. We use weakly supervised classification to try and improve results compared to just using supervised learning.
 

#**B) High Level Goals and Methedology:**

Out high level goal is to use weakly supervised classification to improve our 
prediction ability compared to just training using labeled images. We hope to 
achieve atleast a 5% increase in our test prediction score using clustering 
methods.

Our method is split into two experiments, 
- we first will test to see training
a CNN on N-M labeled images, then use a cluestering method to classify all the 
N images, using the majority label in each cluster as the predicted label
- secondly we will try using the predicted labels from the previous step to 
train a new CNN model, to see if it performs better

Our baseline will be a simple CNN of N+1 (with the K extra classes being labelled as "unknown") to differentiate between unlabeled and labeled classes. Then simply run (some unsupervised model) on the unlabeled data. 

#**C) Team Members and Contributions**

- Hardik Paliwal
  - Created CNN based on VGG11 for training
  - Created function to gather features from pretrained CNN (trained on M labeled images)
  - Created method to get predicted labels from clustering methods

- Lance Pereira
  - (TODO) Modified VGG11 like CNN for better results
  - Created experiments for ratios of M/N 
  - Split data into labeled and unlabeled
  - Created experiments for diffirent clustering methods
    - Kmeans, Kmedians, Kmodes
    - GMM
  - Modified the Clustering method labeler to work with unlabeled data and labeled data
  - Create function to get retrained CNN using clustered labels

#**D) External Code Libraries**

We used

- Pytorch
  - Because it was crucial for the quick training of our models
  - Allowed us to not have to deal with calculating back propogation
- Sklearn
  - Provided us a large range of clustering methods for quick experimentation
- Numpy
  - Useful for large matrix operations

In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.autograd import Variable
import sklearn
from torch import optim
import numpy as np
import itertools

%matplotlib inline

# **E) Code**

### **E) [1] Code: Setup**

The below cells:
- Import Fshion MNIST
- define functions that strip label from data

In [39]:
# Constants
dev=torch.device("cuda") 
NUM_EPOCHS = 6
NUM_CLUSTERS = 30
NUM_CLASSES = 10
UNKNOWN_CLASS = 11

In [40]:
trainset = tv.datasets.FashionMNIST(root="./", download=True,train=True,  transform=tv.transforms.Compose(
    [tv.transforms.Resize(32), tv.transforms.ToTensor()]))
trainloader = DataLoader(trainset, batch_size=128, shuffle=True)

testset = tv.datasets.FashionMNIST(root="./", download=True,train=False,  transform=tv.transforms.Compose(
    [tv.transforms.Resize(32), tv.transforms.ToTensor()]))
testloader = DataLoader(testset, batch_size=128, shuffle=True)

### **E) [2] Code: Define our Base CNN (Based on VGG11), Test, Train methods**

In [41]:
#This is an implementation of VGG11 (which is a precursor to VGG16) for mnist dataset.
# it also takes in n, which is the number of classes. N+1 class stands for unknown. 

#this will let us differeniate the unlabelled data from the labelled data
class BasicNet(nn.Module):
    def __init__(self, n=9):
        super(BasicNet, self).__init__()
        self.batchNorm = [nn.BatchNorm2d(64), nn.BatchNorm2d(128),nn.BatchNorm2d(256), nn.BatchNorm2d(256),
                          nn.BatchNorm2d(512), nn.BatchNorm2d(512), nn.BatchNorm2d(512), nn.BatchNorm2d(512)]
        self.conv = [
        nn.Conv2d(1, 64, 3, 1, 1) ,nn.Conv2d(64, 128, 3, 1, 1), nn.Conv2d(128, 256, 3, 1, 1), nn.Conv2d(256, 256, 3, 1, 1)
       ,nn.Conv2d(256, 512, 3, 1, 1), nn.Conv2d(512, 512, 3, 1, 1), nn.Conv2d(512, 512, 3, 1, 1), nn.Conv2d(512, 512, 3, 1, 1)
        ]
        maxPool = nn.MaxPool2d(2, stride=2)
        self.conv1 = nn.Sequential(self.conv[0], self.batchNorm[0], nn.ReLU(), maxPool)
        self.conv2 = nn.Sequential(self.conv[1], self.batchNorm[1], nn.ReLU(), maxPool)
        self.conv3 = nn.Sequential(self.conv[2], self.batchNorm[2], nn.ReLU())
        self.conv4 = nn.Sequential(self.conv[3], self.batchNorm[3], nn.ReLU(), maxPool) 
        self.conv5 = nn.Sequential(self.conv[4], self.batchNorm[4], nn.ReLU())
        self.conv6 = nn.Sequential(self.conv[5], self.batchNorm[5], nn.ReLU(), maxPool)
        self.conv7 = nn.Sequential(self.conv[6], self.batchNorm[6], nn.ReLU()) 
        self.conv8 = nn.Sequential(self.conv[7], self.batchNorm[7], nn.ReLU(), maxPool)
        self.fc1 = nn.Linear(512, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, n+1)
        
    def forward(self, x, feature_embedding=False):
        dropOut = nn.Dropout(p=0.5)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = torch.flatten(x, 1)
        
        x = dropOut(F.relu(self.fc1(x)))
        x = dropOut(F.relu(self.fc2(x)))
        if(feature_embedding):
          return x
        #Not sure why this works without softmax. probably a reasoning givin in the paper (cause the output can range from anything (not normalized to a 0-1 probability range))
        x = self.fc3(x)
        return x

In [42]:
def test(data, net):
    net.eval()
    loss_func = nn.CrossEntropyLoss()

    total_correct = 0
    total_loss = 0
    with torch.no_grad():
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(data):
            images= images.to(dev)
            labels = labels.to(dev)
            test_pred = net(images)

            pred = torch.max(test_pred, 1)[1].data.squeeze()
            total_correct+= (pred == labels).sum().item()
            loss = loss_func( test_pred, labels)
            total_loss+= loss.item()*images.size(0)
        # return total_correct/len(data.dataset), total_loss/len(data.dataset)
        # Note From Lance: We shouldn't divide the total loss
        return total_correct/len(data.dataset), total_loss
  
def train(num_epochs, net, trainloader):
    optimizer = optim.SGD(net.parameters(), lr=0.01)
    loss_func = nn.CrossEntropyLoss()

    accuracy_through_epochs = []
    total_step = len(trainloader)
    
    for epoch in range(num_epochs):
        net.train()
        for i, (images, labels) in enumerate(trainloader):
            images= images.to(dev)
            labels = labels.to(dev)
            optimizer.zero_grad()           
            prediction = net(images)
            loss = loss_func( prediction, labels)
            loss.backward()
            optimizer.step()
            if ((i +1) % 100 == 0):
                print(f"Epoch {epoch+1} / {num_epochs}, Step {i+1}/ {total_step} , Loss {loss.item()}")

    return accuracy_through_epochs, net

### **E) [3] Code: Split Fashion MNIST training data into M labeled images, and N-M unlabeled**

In [43]:
#Modifies dataset in place to only have values correspounding to the labels in classesToUse
def splitTrainingData(training_data, M_percent):
  # N is len(training_data)
  len_N = len(trainset)

  # M is the number of labeled images we want
  len_M = int(M_percent*len_N)
  
  labeled_data, unlabeled_data = torch.utils.data.random_split(trainset, [len_M, len_N - len_M])

  # strip the labels from unlabeled_data
  # unlabeled_data.dataset.targets[unlabeled_data.indices] = UNKNOWN_CLASS

  labeled_data_loader = DataLoader(labeled_data, batch_size=128, shuffle=True)
  unlabeled_data_loader = DataLoader(unlabeled_data, batch_size=128, shuffle=True)

  return labeled_data_loader, unlabeled_data_loader, labeled_data, unlabeled_data


In [44]:
labeled_train_loader, unlabeled_train_loader, labeled_train_data, unlabeled_train_data = splitTrainingData(trainset, 0.7)

### **E) [4] Code: Train Base CNN on M labeled training images**

In [45]:
net = BasicNet()
net.to(dev)
result, trained_net = train(NUM_EPOCHS, net, labeled_train_loader)

Epoch 1 / 6, Step 100/ 329 , Loss 0.563522219657898
Epoch 1 / 6, Step 200/ 329 , Loss 0.3923516571521759
Epoch 1 / 6, Step 300/ 329 , Loss 0.4406273365020752
Epoch 2 / 6, Step 100/ 329 , Loss 0.3055887818336487
Epoch 2 / 6, Step 200/ 329 , Loss 0.21421709656715393
Epoch 2 / 6, Step 300/ 329 , Loss 0.1780194640159607
Epoch 3 / 6, Step 100/ 329 , Loss 0.4110018312931061
Epoch 3 / 6, Step 200/ 329 , Loss 0.3142193555831909
Epoch 3 / 6, Step 300/ 329 , Loss 0.21675387024879456
Epoch 4 / 6, Step 100/ 329 , Loss 0.27567151188850403
Epoch 4 / 6, Step 200/ 329 , Loss 0.20697882771492004
Epoch 4 / 6, Step 300/ 329 , Loss 0.18259502947330475
Epoch 5 / 6, Step 100/ 329 , Loss 0.11308476328849792
Epoch 5 / 6, Step 200/ 329 , Loss 0.19030247628688812
Epoch 5 / 6, Step 300/ 329 , Loss 0.10715696960687637
Epoch 6 / 6, Step 100/ 329 , Loss 0.18783345818519592
Epoch 6 / 6, Step 200/ 329 , Loss 0.1297719031572342
Epoch 6 / 6, Step 300/ 329 , Loss 0.10071364790201187


### **E) [5] Code: Use the 'semi' trained CNN to gain important features (feature embedings) of N-M *unlabeled* training images**

In [46]:
#do this to store the results in 1 numpy array of 10000 images vs like 20 batches of size 128 images. 
#get memory error when doing it on a batch of size len(trainloader), so we have to combine the results for trainloader

def getFeatureEmbedings(dataloader):
  featureEmbed = []
  predictedDigit = []

  with torch.no_grad():
    for i, (images, labels) in enumerate(dataloader):
        images= images.to(dev)
        labels = labels.to(dev)
        featureEmbed.append(net(images, feature_embedding=True).to("cpu").numpy())
        pred = net(images)
        predictedDigit.append(torch.max(pred, 1)[1].data.squeeze().to("cpu").numpy())

  # flatten lists
  featureEmbed = np.array(list(itertools.chain(*featureEmbed)))
  predictedDigit = np.array(list(itertools.chain(*predictedDigit)))

  return featureEmbed, predictedDigit

In [47]:
# We need the data with the labels
trainFeatureEmbed, trainPredictedDigit = getFeatureEmbedings(trainloader)

### **E) [6] Code: Assign labels to unlabeled images**

In [48]:
#In order to see how well k-means did we can use this supervised method of defining what a cluster is by seting the cluster label as the most common digits in that cluster
#unsupervised approaches include: manually selecting class depending on mean image
def retrieve_cluster_to_classification(cluster_labels,y_train):
  reference_labels = {}
# For loop to run through each label of cluster label
  for i in range(len(np.unique(kmeans.labels_))):
    index = np.where(cluster_labels == i,1,0)
    # we only read 0:NUM_CLASSES so we dont read the unknown labels
    num = np.bincount(y_train[index==1])[:NUM_CLASSES].argmax()
    reference_labels[i] = num
    # TODO: Right now the refrence label just maps to the majority label, should we also consider the 2nd and 3rd highest
  return reference_labels

### **E) [7] Code: Define retrained CNN function using predicted labels from clustering for unlabeled data**



In [49]:
def retrain_CNN_with_predicted_labels(train_dataset, my_labeled_trainset, my_unlabeled_trainset, unlabeled_train_loader, cluster_method):
  
  # remove all unlabeled labels from main training set, set them to NUM_CLASSES
  train_targets = np.array(train_dataset.targets.numpy(), copy=True)  
  train_targets[my_unlabeled_trainset.indices] = NUM_CLASSES
  
  # mapping from NUM_CLUSTERS to FASHION_MNIST classes
  reference_labels = retrieve_cluster_to_classification(
      cluster_method.labels_, 
      train_targets
  )

  unlabelTrainFeatureEmbed, _ = getFeatureEmbedings(unlabeled_train_loader)
  
  # Assign new labels
  predicted_test = cluster_method.predict(unlabelTrainFeatureEmbed)
  
  for i in range(unlabelTrainFeatureEmbed.shape[0]):
    my_unlabeled_trainset.dataset.targets[my_unlabeled_trainset.indices[i]] = reference_labels[predicted_test[i]]

  new_combined_data = torch.utils.data.ConcatDataset([my_labeled_trainset, my_unlabeled_trainset])
  new_combined_data_loader = DataLoader(new_combined_data, batch_size=128, shuffle=True)
  retrained_net = BasicNet()
  retrained_net.to(dev)
  result, retrained_net = train(NUM_EPOCHS, retrained_net, new_combined_data_loader)
  return retrained_net



#**Experiments**


###Experiment A) First we will create three experiments using diffirent ratios of M:N-M


1.   Ratio of 70% labeled to 30% unlabeled
2.   Ratio of 50% labeled to 50% unlabeled
3.   Ratio of 30% labeled to 70% unlabeled




In [50]:
# Experiment A: Diffirent Ratios of M to N

labeled_train_loader_70, unlabeled_train_loader_30, labeled_train_data_70, unlabeled_train_data_30 = splitTrainingData(trainset, 0.7)
labeled_train_loader_50, unlabeled_train_loader_50, labeled_train_data_50, unlabeled_train_data_50 = splitTrainingData(trainset, 0.5)
labeled_train_loader_30, unlabeled_train_loader_70, labeled_train_data_30, unlabeled_train_data_70 = splitTrainingData(trainset, 0.3)

###Experiment B) Then we will try training it with diffirent clustering methods

In [51]:
from sklearn.cluster import MiniBatchKMeans

kmeans = MiniBatchKMeans(n_clusters = NUM_CLUSTERS)

kmeans.fit(trainFeatureEmbed)

MiniBatchKMeans(n_clusters=30)

In [67]:
from sklearn.cluster import DBSCAN

gmm = DBSCAN(eps=3, min_samples= NUM_CLUSTERS)

gmm.fit(trainFeatureEmbed)

DBSCAN(eps=3, min_samples=30)

###Experiment C) Train all the models combining diffirent experiments

In [52]:
retrained_net_kmeans_70_label_30_not = retrain_CNN_with_predicted_labels(trainset, labeled_train_data_70, unlabeled_train_data_30, unlabeled_train_loader_30, kmeans)

Epoch 1 / 6, Step 100/ 469 , Loss 1.5237332582473755
Epoch 1 / 6, Step 200/ 469 , Loss 1.2228803634643555
Epoch 1 / 6, Step 300/ 469 , Loss 1.2317708730697632
Epoch 1 / 6, Step 400/ 469 , Loss 1.4264209270477295
Epoch 2 / 6, Step 100/ 469 , Loss 1.4124491214752197
Epoch 2 / 6, Step 200/ 469 , Loss 1.3803890943527222
Epoch 2 / 6, Step 300/ 469 , Loss 1.104261040687561
Epoch 2 / 6, Step 400/ 469 , Loss 1.4154531955718994
Epoch 3 / 6, Step 100/ 469 , Loss 1.2753548622131348
Epoch 3 / 6, Step 200/ 469 , Loss 1.270864725112915
Epoch 3 / 6, Step 300/ 469 , Loss 1.2200355529785156
Epoch 3 / 6, Step 400/ 469 , Loss 1.3213220834732056
Epoch 4 / 6, Step 100/ 469 , Loss 1.0486948490142822
Epoch 4 / 6, Step 200/ 469 , Loss 1.1233041286468506
Epoch 4 / 6, Step 300/ 469 , Loss 1.1874897480010986
Epoch 4 / 6, Step 400/ 469 , Loss 0.9088383316993713
Epoch 5 / 6, Step 100/ 469 , Loss 1.0206546783447266
Epoch 5 / 6, Step 200/ 469 , Loss 1.306497573852539
Epoch 5 / 6, Step 300/ 469 , Loss 1.0899863243103

In [53]:
retrained_net_kmeans_50_label_50_not = retrain_CNN_with_predicted_labels(trainset, labeled_train_data_50, unlabeled_train_data_50, unlabeled_train_loader_50, kmeans)

Epoch 1 / 6, Step 100/ 469 , Loss 1.3262841701507568
Epoch 1 / 6, Step 200/ 469 , Loss 1.1499066352844238
Epoch 1 / 6, Step 300/ 469 , Loss 1.3281002044677734
Epoch 1 / 6, Step 400/ 469 , Loss 1.1707172393798828
Epoch 2 / 6, Step 100/ 469 , Loss 1.1296488046646118
Epoch 2 / 6, Step 200/ 469 , Loss 1.1957318782806396
Epoch 2 / 6, Step 300/ 469 , Loss 0.9648768901824951
Epoch 2 / 6, Step 400/ 469 , Loss 1.208524465560913
Epoch 3 / 6, Step 100/ 469 , Loss 1.0582894086837769
Epoch 3 / 6, Step 200/ 469 , Loss 1.2255021333694458
Epoch 3 / 6, Step 300/ 469 , Loss 1.0864713191986084
Epoch 3 / 6, Step 400/ 469 , Loss 1.1648823022842407
Epoch 4 / 6, Step 100/ 469 , Loss 1.0826817750930786
Epoch 4 / 6, Step 200/ 469 , Loss 1.306212306022644
Epoch 4 / 6, Step 300/ 469 , Loss 0.9369737505912781
Epoch 4 / 6, Step 400/ 469 , Loss 0.9683341979980469
Epoch 5 / 6, Step 100/ 469 , Loss 1.0869994163513184
Epoch 5 / 6, Step 200/ 469 , Loss 1.0911811590194702
Epoch 5 / 6, Step 300/ 469 , Loss 1.217448234558

In [54]:
retrained_net_kmeans_30_label_70_not = retrain_CNN_with_predicted_labels(trainset, labeled_train_data_30, unlabeled_train_data_70, unlabeled_train_loader_70, kmeans)

Epoch 1 / 6, Step 100/ 469 , Loss 0.7395923137664795
Epoch 1 / 6, Step 200/ 469 , Loss 0.49900320172309875
Epoch 1 / 6, Step 300/ 469 , Loss 0.6023542284965515
Epoch 1 / 6, Step 400/ 469 , Loss 0.597287118434906
Epoch 2 / 6, Step 100/ 469 , Loss 0.767909586429596
Epoch 2 / 6, Step 200/ 469 , Loss 0.5157312154769897
Epoch 2 / 6, Step 300/ 469 , Loss 0.4789220690727234
Epoch 2 / 6, Step 400/ 469 , Loss 0.333807110786438
Epoch 3 / 6, Step 100/ 469 , Loss 0.4427388608455658
Epoch 3 / 6, Step 200/ 469 , Loss 0.5311180949211121
Epoch 3 / 6, Step 300/ 469 , Loss 0.509806215763092
Epoch 3 / 6, Step 400/ 469 , Loss 0.7157886624336243
Epoch 4 / 6, Step 100/ 469 , Loss 0.63088059425354
Epoch 4 / 6, Step 200/ 469 , Loss 0.472177654504776
Epoch 4 / 6, Step 300/ 469 , Loss 0.6406874060630798
Epoch 4 / 6, Step 400/ 469 , Loss 0.46609607338905334
Epoch 5 / 6, Step 100/ 469 , Loss 0.5201724767684937
Epoch 5 / 6, Step 200/ 469 , Loss 0.4584900438785553
Epoch 5 / 6, Step 300/ 469 , Loss 0.529569804668426

In [68]:
retrained_net_gmm_70_label_30_not = retrain_CNN_with_predicted_labels(trainset, labeled_train_data_70, unlabeled_train_data_30, unlabeled_train_loader_30, gmm)

ValueError: ignored

In [74]:
len(gmm.labels_)

60000

In [57]:
retrained_net_gmm_50_label_50_not = retrain_CNN_with_predicted_labels(trainset, labeled_train_data_50, unlabeled_train_data_50, unlabeled_train_loader_50, gmm)

In [58]:
retrained_net_gmm_30_label_70_not = retrain_CNN_with_predicted_labels(trainset, labeled_train_data_30, unlabeled_train_data_70, unlabeled_train_loader_70, gmm)

#**Results**


###First we will look at the **baseline** results from the CNN trained on M labeled images

In [59]:
testFeatureEmbed, _ = getFeatureEmbedings(testloader)

In [60]:
baseline_accuracy, loss = test(testloader, net)
print(f"Our accuracy with just the CNN is: {baseline_accuracy} on test set from training on trainset" )

Our accuracy with just the CNN is: 0.8524 on test set from training on trainset


### Next we will look at all the other methods that use weakly supervised learning, we aim to be better than the baseline in each


In [61]:
accuracy_kmeans_70_label_30_not, loss = test(testloader, retrained_net_kmeans_70_label_30_not)
print(f"Our accuracy with the CNN and Kmeans clustering with 70% labeled, 30% unlabeled is: {accuracy_kmeans_70_label_30_not} on test set" )

Our accuracy with the CNN and Kmeans clustering with 70% labeled, 30% unlabeled is: 0.8694 on test set from training on trainset


In [62]:
accuracy_kmeans_50_label_50_not, loss = test(testloader, retrained_net_kmeans_50_label_50_not)
print(f"Our accuracy with the CNN and Kmeans clustering with 50% labeled, 50% unlabeled is: {accuracy_kmeans_50_label_50_not} on test set" )

Our accuracy with the CNN and Kmeans clustering with 50% labeled, 50% unlabeled is: 0.1273 on test set from training on trainset


In [63]:
accuracy_kmeans_30_label_70_not, loss = test(testloader, retrained_net_kmeans_30_label_70_not)
print(f"Our accuracy with the CNN and Kmeans clustering with 30% labeled, 70% unlabeled is: {accuracy_kmeans_30_label_70_not} on test set" )

Our accuracy with the CNN and Kmeans clustering with 30% labeled, 70% unlabeled is: 0.1 on test set from training on trainset


In [64]:
# accuracy_gmm_70_label_30_not, loss = test(testloader, retrained_net_gmm_70_label_30_not)
# print(f"Our accuracy with the CNN and GMM clustering with 70% labeled, 30% unlabeled is: {accuracy_gmm_70_label_30_not} on test set from training on trainset" )

In [65]:
# accuracy_gmm_50_label_50_not, loss = test(testloader, retrained_net_gmm_50_label_50_not)
# print(f"Our accuracy with the CNN and GMM clustering with 50% labeled, 50% unlabeled is: {accuracy_gmm_50_label_50_not} on test set from training on trainset" )

In [66]:
# accuracy_gmm_30_label_70_not, loss = test(testloader, retrained_net_gmm_30_label_70_not)
# print(f"Our accuracy with the CNN and GMM clustering with 30% labeled, 70% unlabeled is: {accuracy_gmm_30_label_70_not} on test set from training on trainset" )

#**Conclusions**

- We found that the most useful ratio of M/N which was 30% labeled, 70% unlabeled yieled
- This compared to 50:50
- Compared to 70% labeled, 30% unlabeled